In [169]:
import mlflow
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [170]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("baseline_model_experiment")

DEBUG:urllib3.connectionpool:Resetting dropped connection: 127.0.0.1
DEBUG:urllib3.connectionpool:http://127.0.0.1:5000 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=baseline_model_experiment HTTP/1.1" 200 258


<Experiment: artifact_location='s3://mlops-exp-models/models/1', creation_time=1717962983825, experiment_id='1', last_update_time=1717962983825, lifecycle_stage='active', name='baseline_model_experiment', tags={}>

In [171]:
cleaned_data= pd.read_csv('../../data/cleaned_new_york_air_quality.csv')
cleaned_data.head()

,Unnamed: 0,date,pm25,o3,no2,co
0,0,2024-05-01,54.0,30.0,16.0,2.0
1,3,2024-05-04,27.0,22.0,10.0,2.0
2,5,2024-05-06,66.0,44.0,12.0,2.0
3,6,2024-05-07,34.0,41.0,18.0,2.0
4,8,2024-05-09,17.0,23.0,8.0,2.0


In [172]:
for i in range(1, 8):
    cleaned_data[f'pm25_lag_{i}'] = cleaned_data['pm25'].shift(i)
cleaned_data.dropna(inplace=True)
cleaned_data.head()

,Unnamed: 0,date,pm25,o3,no2,co,pm25_lag_1,pm25_lag_2,pm25_lag_3,pm25_lag_4,pm25_lag_5,pm25_lag_6,pm25_lag_7
7,13,2024-05-14,36.0,30.0,13.0,2.0,39.0,20.0,17.0,34.0,66.0,27.0,54.0
8,16,2024-05-17,29.0,33.0,10.0,2.0,36.0,39.0,20.0,17.0,34.0,66.0,27.0
9,18,2024-05-19,18.0,30.0,11.0,2.0,29.0,36.0,39.0,20.0,17.0,34.0,66.0
10,19,2024-05-20,30.0,34.0,17.0,2.0,18.0,29.0,36.0,39.0,20.0,17.0,34.0
11,20,2024-05-21,39.0,40.0,14.0,2.0,30.0,18.0,29.0,36.0,39.0,20.0,17.0


In [173]:
X = cleaned_data[[f'pm25_lag_{i}' for i in range(1, 8)]]
y = cleaned_data['pm25']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [174]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
with mlflow.start_run() as run:
    mlflow.log_param("fit_intercept", model.fit_intercept)
    mlflow.log_metric("train_score", model.score(X_train, y_train))
    mlflow.log_metric("test_score", model.score(X_test, y_test))
    mlflow.sklearn.log_model(model, "model")

    run_info = mlflow.active_run().info
    artifact_uri = run_info.artifact_uri
    print(f"Run ID: {run_info.run_id}")
    print(f"Artifact location: {artifact_uri}")